# Kryptografia asymetryczna - kryptosystem RSA 
Kryptografia asymetryczna charakteryzuje się wykorzystaniem **pary kluczy publiczny-prywatny** (stąd nazwa kryptografia z kluczem publicznym). Klucz publiczny może być swobodnie dystrybuowany otwartym kanałem i służy do szyfrowania (a także do weryfikowania podpisu). Klucz prywatny musi być utrzymywany w tajności i służy do deszyfrowania (lub tworzenia podpisu). 

Chronologicznie pierwszym kryptosystemem asymetrycznym był protokół wymiany kluczu Diffiego-Hellmana-Merkla. Służy on bezpiecznej wymiany danych, które mogą być wykorzystane jako tajne klucze kryptograficzne lub mogą być użyte do wyprodukowania kluczy. 

Najbardziej znanym kryptosystem z kluczem publicznym jest RSA (nazwa pochodzi od wynalazów: Rivest, Shamir i Adlemann). RSA umożliwia szyfrowanie danych jak również realizację podpisu cyfrowego. Bezpieczeństwo RSA opiera się na obliczeniowej trudności rozwiązania **problemu faktoryzacji liczb całkowitych złożonych**. 

## Generowanie kluczy w kryptosystemie RSA

### 1. Losujemy dwie duże liczby pierwsze 
Potrzebujemy dwóch liczb pierwszych o naprawdę dużych rozmiarach - 2048 bitów obecnie uważa się niezbyt bezpieczny wybór. 4096 bitów jest z kolei wielkością nieco kłopotliwą w użytkowaniu. 
#### Skąd wziąć liczbę pierwszą? 
**Wylosować i sprawdzić czy jest pierwsza!**


Test probabilistyczny, np. Rabina-Millera. **(A to już znamy!!!)**

## Zadanie
1. Napisz funkcję generującą liczbę pierwszą o określonej długości w bitach. 

In [3]:
import random

def miller_rabin(n, k=40):
        if n < 2:
            return False
        if n == 2 or n == 3:
            return True
        if n % 2 == 0:
            return False
        
        r, d = 0, n - 1
        while d % 2 == 0:
            r += 1
            d //= 2
        
        for _ in range(k):
            a = random.randrange(2, n - 1)
            x = pow(a, d, n)
            
            if x == 1 or x == n - 1:
                continue
            
            for _ in range(r - 1):
                x = pow(x, 2, n)
                if x == n - 1:
                    break
            else:
                return False
        
        return True

In [4]:
def generatePrime(keysize):
    while True:
        num = random.getrandbits(keysize)
        num |= (1 << keysize - 1) | 1  # nieparzystosc i dlugosc
        if miller_rabin(num): return num

### 2. Obliczamy składniki kluczy 
1. Wybieramy dwie duże liczby pierwsze $p$ i $q$
2. Pierwszym składnikiem klucza jest moduł $n$ $n=p \times q$ 
3. Poszukujemy wykładnika publicznego $e$, który jest względnie pierwszy z $(p-1)\cdot (q-1)$ (czasami używane jest w miejscu pojęcie tocjentu lub funkcji Eulera: $\phi(n) = \phi(p)\cdot \phi(q) = (p − 1)·(q − 1)$)
4. Poszukujemy wykładnika prywatnego $d$, które jest odwrotnością $e\ (mod\ (p-1)\cdot (q-1))$: $de \equiv 1  (mod\ (p-1)\cdot (q-1))$ **(potrzebujemy rozszerzonego algorytmu Euklidesa!!!)**
5. Kluczem publiczny jest para $(n, e)$, kluczem prywatnym jest para $(n, d)$.

## Zadanie 

1. Napisz funkcję generującą klucze RSA o ustalonym rozmiarze

In [5]:
from sympy import gcd

def generateKey(keySize):
    p, q = generatePrime(keySize), generatePrime(keySize)
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    while gcd(e, phi) != 1: e += 2
    d = pow(e, -1, phi)
    publicKey = (n, e)
    privateKey = (n, d)
    return (publicKey, privateKey)

def makeKeyFiles(keySize):
    public, private = generateKey(keySize)
    print('pub', public,"\n")
    print('priv', private)



In [6]:
makeKeyFiles(24)

pub (175660724604887, 65537) 

priv (175660724604887, 166585104796841)


## Zadanie 

Napisz funkcje implementujące szyfrowanie i deszyfrowanie RSA (tzw. podręcznikowe)

### Szyfrowanie RSA 
Operacja szyfrowania: $c=m^e (mod\ n)$

In [2]:
def encrypt(message, mod, exp):
    message_ascii = [ord(ch) for ch in message] 
    encrypted = [pow(m, exp, mod) for m in message_ascii] 
    #encrypted = [m**exp%mod for m in message_ascii] 
    return encrypted


### Deszyfrowanie RSA 
Operacja szyfrowanie $m = c^d (mod\ n)$

In [1]:
def decrypt(message_encrypted, mod, exp):
    message_ascii = [pow(c, exp, mod) for c in message_encrypted]
    #message_ascii = [c**exp%mod for c in message_encrypted]
    return ''.join(chr(m) for m in message_ascii)


In [8]:
A = "Hello world!"
public, private = generateKey(256)
message_encrypted = encrypt(A, public[0], public[1])
print("Encrypted message:", message_encrypted)
message_decrypted = decrypt(message_encrypted, private[0], private[1])
print("Decrypted message:", message_decrypted)

Encrypted message: [2091098397538915405945297018439806140356556519151556633781615939706716823777060399415057220986224108729937874752615631422863793553790239613801817087186123, 1211875158302233458382978019532070720163323831937617982938372266895099496301229034093508538312240902254758159974462297868542365443247365771092331506441189, 7041343249892350359372291709355016843217965658055050877018639711425445517238222941436465471828684542881217372760920866601373912423317199493053985081129405, 7041343249892350359372291709355016843217965658055050877018639711425445517238222941436465471828684542881217372760920866601373912423317199493053985081129405, 529543829257585038077172578789366342851652441221045759617739629807358944751641855077374184562614127053255466397930239718947169798694762076464171546724702, 1608707029952657480970241491184037735168135098069210204956601143446329357706003836859368350757601141432964862592037567377448343571669358007843700373456452, 614904391682029301904361058951069183544114322

## Zastanów się
1. Sprawdź działanie powyższej implementacji dla różnych wielkości klucza (podawane podczas generowania kluczy) - zweryfikuj jak na wydajnosć wpływa zastosowanie różnych sposobów potęgowania dostępnych w Python i jego bibliotekach.  
2. Poszukaj informacji o trybie podręcznikowym RSA (*textbook RSA encryption*). Na czym polega? Jakie są jego wady i zalety? 


### Algorytm szybkiego potęgowania 
1. Zwykłe potęgowanie $n^{exp}$: $exp$ mnożeń 
2. Algorytm szybkiego potęgowania: część mnożeń zastępujemy podnoszeniem do kwadratu (_squaring_).
    __Skąd mamy wiedzieć kiedy mnożyć, a kiedy potęgować?__

In [12]:
def fastModularExponentation(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m